# Design

Funtionality:
- Load data
- Run interpolation
- Return relevant data

Attributes:
- ZA specs (min/max)
- Nscale specs
- Map specs (size)

Methods:
- load_amc()
- interpolate()
- visualize()

In [1]:
from src.ozone import Ozone

df = Ozone(nscale=(0,1,2), airmass=(3,4,5))

In [8]:
df.nscale.points = 9
df.nscale.points

9